In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/airesidency/sample.csv
/kaggle/input/airesidency/train.csv
/kaggle/input/airesidency/test.csv


## 1. Import, View data

In [2]:
train_df = pd.read_csv('../input/airesidency/train.csv', sep=';')

# sample_df = pd.read_csv('../input/airesidency/sample.csv',on_bad_lines='skip')

train_df.columns

Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque',
       'seats'],
      dtype='object')

In [3]:
train_df

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Tata Nexon 1.5 Revotorq XM,2018,850000,9500,Diesel,Individual,Manual,First Owner,21.5 kmpl,1497 CC,108.5 bhp,260Nm@ 1500-2750rpm,5.0
1,Hyundai i20 Active SX Diesel,2018,750000,30000,Diesel,Individual,Manual,First Owner,21.19 kmpl,1396 CC,88.76 bhp,219.66nm@ 1500-2750rpm,5.0
2,Toyota Fortuner 4x2 AT,2015,1500000,80000,Diesel,Individual,Automatic,Second Owner,12.55 kmpl,2982 CC,168.5 bhp,360Nm@ 1400-3200rpm,7.0
3,Hyundai Verna 1.6 VTVT SX,2015,795000,35000,Petrol,Individual,Manual,First Owner,17.01 kmpl,1591 CC,121.3 bhp,154.9Nm@ 4200rpm,5.0
4,Volvo XC40 D4 Inscription BSIV,2019,3800000,20000,Diesel,Individual,Automatic,First Owner,18.0 kmpl,1969 CC,190 bhp,400Nm,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Maruti A-Star Lxi,2010,235000,50000,Petrol,Individual,Manual,First Owner,19.0 kmpl,998 CC,66.1 bhp,90Nm@ 3500rpm,5.0
5996,Maruti Wagon R LXI CNG,2018,390000,40000,CNG,Individual,Manual,First Owner,26.6 km/kg,998 CC,58.16 bhp,77Nm@ 3500rpm,5.0
5997,Tata Tigor 1.2 Revotron XT,2018,450000,75000,Petrol,Individual,Manual,First Owner,20.3 kmpl,1199 CC,84 bhp,114Nm@ 3500rpm,5.0
5998,Mahindra Bolero 2011-2019 DI NON AC BS III White,2011,250000,100000,Diesel,Individual,Manual,Second Owner,15.96 kmpl,2523 CC,63 bhp,180Nm@ 1440-1500rpm,9.0


In [4]:
## EXTRACT `COMPANY` FROM `NAME` 
# names = train_df['name'].str.split()
# # company = i[0] for i in names
# company = []
# for i in names:
#     company.append(i[0])
    
# company
# train_df['company'] = company
# train_df
# len(train_df['company'].unique())

In [5]:
def correct_mileage(df): # to be used in the clean_cols function
    Correct_Mileage= []
    for i in df.mileage:
        if str(i).endswith('km/kg'):
            i = i[:-6]
            i = float(i)*1.40
            Correct_Mileage.append(float(i))
        elif str(i).endswith('kmpl'):
            i = i[:-6]
            #print(i)
            Correct_Mileage.append(float(i))
        else:
            Correct_Mileage.append(Correct_Mileage[-1])
    return Correct_Mileage

In [6]:
def create_dummies(df, col): # to be used in the clean_cols function
    dummies = pd.get_dummies(df[col])
    df = df.drop(col, axis=1)
    df = df.join(dummies)
    return df

In [7]:
def correct_torque(df):  # to be used in the clean_cols function
    Correct_Torque= []
    for i in df.torque:
        if str(i).endswith('NM') or str(i).endswith('nM') or str(i).endswith('Nm')  or str(i).endswith('nm') :
            Correct_Torque.append(i)
        elif str(i).endswith('kgm')or str(i).endswith('KGM'):
            i = i[:-3]
            #print(i)
            Correct_Torque.append(i)
        else:
            Correct_Torque.append(Correct_Torque[-1])
    return Correct_Torque

In [8]:
def clean_cols(df):
    # fillna 
    df['max_power'] = df['max_power'].fillna(df['max_power'].mode()[0])    
    df['engine'] = df['engine'].fillna(df['engine'].mode()[0])    
    df['mileage'] = df['mileage'].fillna(df['mileage'].mode()[0])    
    df['seats'] = df['seats'].fillna(df['seats'].mean())
    
    engine = df['engine'].astype(str).map(lambda x: x.rstrip(' C'))
    df['engine'] = engine.astype(float)
    
    df['max_power'] = pd.to_numeric(df['max_power'].str.replace(" bhp", ""))
    df['max_power'] = df['max_power'].fillna(df['max_power'].mean())
#     df['max_power'].isnull().sum()


    df['mileage']=correct_mileage(df)

    df['transmission'].replace({'Manual': False,'Automatic': True}, inplace=True)
    df = create_dummies(df, 'fuel')
    df = create_dummies(df, 'seller_type')
    df = create_dummies(df, 'owner')

    
    df['torque'] = df['torque'].str.split('@',expand=True)[0]  #eliminate the rpm where max-torque can be reached
    df['torque'] = df['torque'].str.split('at',expand=True)[0]  #eliminate the rpm where max-torque can be reached
    df['torque'] = df['torque'].str.split('/',expand=True)[0]  #eliminate the rpm where max-torque can be reached
    
    df['torque'] = correct_torque(df)
    df['torque'] = pd.to_numeric(df['torque'].map(lambda x: x.rstrip(' nNmM')))
    
    df['year'] = pd.to_numeric(df['year'])
#     df['selling_price'] = pd.to_numeric(df['selling_price'])
    df['km_driven'] = pd.to_numeric(df['km_driven'])
    return df

In [9]:
print('number of years covered: ', len(train_df['year'].unique()))

number of years covered:  28


*Should we treat the `year` column as a quantitative column or change it to a categorical column. If then, how can we fit it into the regression model?*

In [10]:
train_df = clean_cols(train_df)
train_df['selling_price'] = pd.to_numeric(train_df['selling_price']) 
    # the clean function works for the test_x also, so it cannot handle the output column
train_df

,name,year,selling_price,km_driven,transmission,mileage,engine,max_power,torque,seats,...,LPG,Petrol,Dealer,Individual,Trustmark Dealer,First Owner,Fourth & Above Owner,Second Owner,Test Drive Car,Third Owner
0,Tata Nexon 1.5 Revotorq XM,2018,850000,9500,False,21.00,1497.0,108.50,260.00,5.0,...,0,0,0,1,0,1,0,0,0,0
1,Hyundai i20 Active SX Diesel,2018,750000,30000,False,21.10,1396.0,88.76,219.66,5.0,...,0,0,0,1,0,1,0,0,0,0
2,Toyota Fortuner 4x2 AT,2015,1500000,80000,True,12.50,2982.0,168.50,360.00,7.0,...,0,0,0,1,0,0,0,1,0,0
3,Hyundai Verna 1.6 VTVT SX,2015,795000,35000,False,17.00,1591.0,121.30,154.90,5.0,...,0,1,0,1,0,1,0,0,0,0
4,Volvo XC40 D4 Inscription BSIV,2019,3800000,20000,True,18.00,1969.0,190.00,400.00,5.0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Maruti A-Star Lxi,2010,235000,50000,False,19.00,998.0,66.10,90.00,5.0,...,0,1,0,1,0,1,0,0,0,0
5996,Maruti Wagon R LXI CNG,2018,390000,40000,False,37.24,998.0,58.16,77.00,5.0,...,0,0,0,1,0,1,0,0,0,0
5997,Tata Tigor 1.2 Revotron XT,2018,450000,75000,False,20.00,1199.0,84.00,114.00,5.0,...,0,1,0,1,0,1,0,0,0,0
5998,Mahindra Bolero 2011-2019 DI NON AC BS III White,2011,250000,100000,False,15.90,2523.0,63.00,180.00,9.0,...,0,0,0,1,0,0,0,1,0,0


In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  6000 non-null   object 
 1   year                  6000 non-null   int64  
 2   selling_price         6000 non-null   int64  
 3   km_driven             6000 non-null   int64  
 4   transmission          6000 non-null   bool   
 5   mileage               6000 non-null   float64
 6   engine                6000 non-null   float64
 7   max_power             6000 non-null   float64
 8   torque                6000 non-null   float64
 9   seats                 6000 non-null   float64
 10  CNG                   6000 non-null   uint8  
 11  Diesel                6000 non-null   uint8  
 12  LPG                   6000 non-null   uint8  
 13  Petrol                6000 non-null   uint8  
 14  Dealer                6000 non-null   uint8  
 15  Individual           

In [12]:
train_df.isnull().sum()

name                    0
year                    0
selling_price           0
km_driven               0
transmission            0
mileage                 0
engine                  0
max_power               0
torque                  0
seats                   0
CNG                     0
Diesel                  0
LPG                     0
Petrol                  0
Dealer                  0
Individual              0
Trustmark Dealer        0
First Owner             0
Fourth & Above Owner    0
Second Owner            0
Test Drive Car          0
Third Owner             0
dtype: int64

## 2. Plotting (if needed)

## 3. Build ML Model, Evaluate

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor

In [14]:
train_x = train_df.drop(['name','selling_price'], axis=1)
train_y = train_df['selling_price']
# train_x.head()

In [15]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2)
# x_test.head()

In [16]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train)

x_test = scaler.transform(x_test)
x_test = pd.DataFrame(x_test)

In [17]:
def choose_model(model,x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    
    y_pred = model.predict(x_test)
    print(model, ': ', r2_score(y_test, y_pred)*100)

In [18]:
models = [LinearRegression(), RandomForestRegressor(),GradientBoostingRegressor(), ElasticNet(), SGDRegressor(), 
          SVR(), LGBMRegressor(), XGBRegressor(), KernelRidge(), CatBoostRegressor(), BayesianRidge()]

# for model in models:
#     choose_model(model, x_train, y_train, x_test, y_test)
"""
LinearRegression()            :  69.2257634658105
RandomForestRegressor()       : *97.24647771100324
GradientBoostingRegressor()   :  96.32895417913836
ElasticNet()                  :  65.96430194659088
SGDRegressor()                :  69.24396850407688
SVR()                         :  -6.134803613175466
LGBMRegressor()               :  97.12046589150343
XGBRegressor                  : *97.88731455224878
KernelRidge()                 :  11.760389563391193
CatBoostRegressor             :  97.78103976436073
BayesianRidge()               :  69.20008943409499
""" 


'\nLinearRegression()            :  69.2257634658105\nRandomForestRegressor()       : *97.24647771100324\nGradientBoostingRegressor()   :  96.32895417913836\nElasticNet()                  :  65.96430194659088\nSGDRegressor()                :  69.24396850407688\nSVR()                         :  -6.134803613175466\nLGBMRegressor()               :  97.12046589150343\nXGBRegressor                  : *97.88731455224878\nKernelRidge()                 :  11.760389563391193\nCatBoostRegressor             :  97.78103976436073\nBayesianRidge()               :  69.20008943409499\n'

Note: r2_score of `RandomForestRegressor` is 96.5% so we can use it as the model to predict our result

## 4. Test-submit

In [19]:
test_x = pd.read_csv('../input/airesidency/test.csv', sep=';')

In [20]:
test_x = clean_cols(test_x)
test_x = test_x.drop(['id','name'], axis=1)

In [21]:
# add a column to fit with the train dataset
test_x['Test Drive Car'] = 0

cols = pd.DataFrame(test_x).columns.tolist()
tmp = cols[-1]
cols[-1] = cols[-2]
cols[-2] = tmp

test_x = test_x[cols]
# cols

In [22]:
 print(pd.DataFrame(test_x).columns == train_x.columns)  #compare columns of train&test

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]


In [23]:
test_x = scaler.transform(test_x)

In [24]:
model = RandomForestRegressor()
model.fit(train_x, train_y)        # the complete dataset from train.csv
y_pred = model.predict(test_x)
y_pred

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


array([63093.96, 61367.92, 61367.92, ..., 68120.  , 68120.  , 62143.96])

In [25]:
y_submit = pd.DataFrame(y_pred)
y_submit.insert(0, 'Id', range(1, 1+len(y_submit)))

y_submit.rename(columns={0:'Expected'}, inplace=True)
y_submit

,Id,Expected
0,1,63093.96
1,2,61367.92
2,3,61367.92
3,4,70241.93
4,5,71017.97
...,...,...
2123,2124,62143.96
2124,2125,68120.00
2125,2126,68120.00
2126,2127,68120.00


In [26]:
## export to .csv file
# y_submit.to_csv('preds-AIREsidencyPractice12.csv',index=False)